In [1]:
import jsockets
import sys
import os
import threading
import filecmp
import time
import matplotlib.pyplot as plt

In [2]:
def execute(filename,packetSize,filesize=0):
    HOST = "anakena.dcc.uchile.cl"
    PORT = 1818
    TIMEOUT_SECONDS = 3

    def Rdr(s):
        f = open(file="resultado.txt", mode="w+")  # [, newline='']

        initial_time = time.perf_counter()
        while True:
            try:
                # max_packet_size debe ser una variable global para el interprete a la hora de invocar a Rdr en su thread.
                data = s.recv(int(max_packet_size)).decode()
            except:
                data = None
            if not data:
                break
            # print(f"Servidor responde: {data}")
            f.write(data)
            s.settimeout(TIMEOUT_SECONDS)
        final_time = time.perf_counter()
        f.close()

        delta_time = final_time - initial_time - TIMEOUT_SECONDS
        #print(f"Tiempo que tomó: {delta_time} (segundos)")

        # 
        if filecmp.cmp(filename, "resultado.txt"):
            print(f"0,{int(max_packet_size)},{delta_time},{filesize}")
            # print("Los archivos son iguales.")  
        else:
            #print("Los archivos son distintos.")
            original_size = os.stat(filename).st_size
            resultado_size = os.stat("resultado.txt").st_size

            loss = original_size - resultado_size

            # print(f'Hay una diferencia de {loss} en bytes entre la respuesta obtenida por el servidor y el archivo original.')
            print(f"{loss},{int(max_packet_size)},{delta_time},{filesize}")

    s = jsockets.socket_udp_connect(HOST, PORT)
    if s is None:
        print('could not open socket')
        sys.exit(1)

    # Envíamos 'Hola' encodeado en bytes para realizar el Handshake.
    s.send("Hola".encode())
    data = s.recv(1024).decode()    # Recibimos respuesta del server

    #print(f"Cliente envía: Hola")
    #print(f"Servidor responde: {data}")

    packet_size = packetSize.zfill(5).encode()
    s.send(packet_size)   # Envíamos propuesta de paquetes
    data = s.recv(1024).decode()            # Recibimos respuesta del server
    max_packet_size = data

    #print(f"Cliente envía: {packet_size.decode()}")
    #print(f"Servidor responde: {data}")

    # Creo thread que lee desde el socket hacia stdout:
    newthread = threading.Thread(target=Rdr, args=(s,))
    newthread.start()

    # En este otro thread leo desde stdin hacia socket:
    f = open(filename, 'r')

    while True:
        buf = f.buffer.read1(int(max_packet_size))
        if not buf:
            break
        s.send(buf)
        #print(f"Cliente envía: {buf.decode()}")

    newthread.join()
    # print("Thread termina")
    # --------------------------------------

In [4]:
print("loss,max_packet_size,time,filesize")
packetSizes = [10,100,200,300,400,500,600,700,800,900,1000,5000,10000,15000,18000,21000,25000,28000,31000,35000,39000,41500,45000,47600,50000]
for x in packetSizes:
    execute('paquetes_1000.txt', str(x),1000)


loss,max_packet_size,time,filesize
0,10,3.041812499999992,1000
0,100,3.0270658000000026,1000
0,200,3.0253010999999788,1000
0,300,3.0283165999999824,1000
0,400,3.0241757999999663,1000
0,500,3.0267120999999975,1000
0,600,3.0341220999999905,1000
0,700,3.029988199999991,1000
0,800,3.023997799999961,1000
0,900,3.0296491000000287,1000
0,1000,3.0249838000000295,1000
0,5000,3.0314632999999844,1000
0,10000,3.020426499999985,1000
0,15000,3.020679799999982,1000
0,18000,3.029828699999996,1000
0,21000,3.0339856999999597,1000
0,25000,3.03349019999996,1000
0,28000,3.027627300000006,1000
0,31000,3.026711799999987,1000
0,35000,3.0794850999999994,1000
0,39000,3.0318568999999798,1000
0,41500,3.0280581000000097,1000
0,45000,3.0300239000000033,1000
0,47600,3.017499500000042,1000
0,50000,3.023087700000019,1000
